In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model

from data_generator import DataGenerator

In [2]:
label_name = {}
for index, folder in enumerate(os.listdir(f'images/train')):
    label_name[folder] = index-1

In [3]:
test_paths = np.array([])
test_labels = {}
for folder in os.listdir('images/test'):
    for file in os.listdir(f'images/test/{folder}'):
        test_paths = np.append(test_paths, np.array([f'images/test/{folder}/{file}']))
        test_labels[f'images/test/{folder}/{file}'] = label_name[folder]

In [4]:
test_generator = DataGenerator(test_paths, test_labels, batch_size=30, shuffle=False)

In [5]:
model_without_maxpooling = load_model(r'models/2.h5')

In [6]:
model_with_maxpooling = load_model(r'models/2_max_pooling.h5')

In [ ]:
model_without_maxpooling.evaluate(test_generator)

In [ ]:
model_with_maxpooling.evaluate(test_generator)

The model which uses convolutional layers has 00.06% points higher accuracy. It's cause convolutional layer can learn how to pool image for certain problem. The model which uses max pooling is faster to learn and has fewer parameters - with many max pooling layers it can be important.

In [9]:
labels = []
for d, l in test_generator:
    labels.append(l)

In [10]:
labels = np.array(labels)
labels = np.reshape(labels, (labels.shape[0]*labels.shape[1],labels.shape[2]))

In [11]:
actual_value = np.argmax(labels, axis=1)

In [ ]:
predicted = np.argmax(model_without_maxpooling.predict(test_generator), axis=1)

In [ ]:
print(classification_report(actual_value, predicted))

In [ ]:
cm = confusion_matrix(actual_value, predicted)
cm_df = pd.DataFrame(cm,
                     index=os.listdir(r'images/test'),
                     columns=os.listdir(r'images/test'))

plt.figure(figsize=(25, 25))
sns.heatmap(cm_df, annot=True, fmt=".10g")
plt.rc("font", size=20)
plt.rc("axes", titlesize=12)
plt.rc("axes", labelsize=12)
plt.rc("xtick", labelsize=12)
plt.rc("ytick", labelsize=12)
plt.rc("legend", fontsize=1)
plt.title("Confusion matrix", fontsize=20)
plt.ylabel("True")
plt.xlabel("Predicted")
plt.show()

In [ ]:
predicted = model_with_maxpooling.predict(test_generator)

In [39]:
predicted = np.argmax(predicted, axis=1)

In [ ]:
print(classification_report(actual_value, predicted))

In [41]:
cm = confusion_matrix(actual_value, predicted)

In [42]:
cm_df = pd.DataFrame(cm,
                     index=os.listdir(r'images/test'),
                     columns=os.listdir(r'images/test'))

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(cm_df, annot=True, fmt=".10g")
plt.rc("font", size=20)
plt.rc("axes", titlesize=12)
plt.rc("axes", labelsize=12)
plt.rc("xtick", labelsize=12)
plt.rc("ytick", labelsize=12)
plt.rc("legend", fontsize=1)
plt.title("Confusion matrix", fontsize=20)
plt.ylabel("True")
plt.xlabel("Predicted")
plt.show()

There are some bad predicted images, but that is only small amount - almost meaningless.